In [3]:
# from pymongo import MongoClient
import numpy as np
import pandas as pd
import datetime
%load_ext line_profiler
pd.options.mode.chained_assignment = None
from datetime import timedelta
import sys
sys.path.insert(0, 'E:/Credence/Projects/Ideal6-python-worker/iModules/returns/Src/')
import dbUnit as db
import computeUnit as cu

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler
database:27017


In [2]:
histData = db.getDataFromMongo("rawdatareturns_navport")
if not histData.empty:
    print("notEmpty")
    # drop a day records
    histData = histData.set_index('ASONDATE').stack().reset_index().rename(columns={0:"VALUE","level_1":"PORTCODE"})
    histData = histData[histData["ASONDATE"] != "04-09-2018"]
    # get daily record
    dailyData = db.getDataFromOracle("nav_daily", ("04-09-2018"))
    rawBench = histData.append(dailyData, ignore_index=True)
else:
    print("Empty")
    histData = db.getDataFromOracle("nav_all")
    # Save all historical data into Mongo
    rawBench = histData
    
rawBench.PORTCODE = rawBench.PORTCODE.astype(str)
rawBench = rawBench.pivot_table(index='ASONDATE',columns='PORTCODE',values='VALUE').reset_index()
db.insertDataToMongo("rawdatareturns_navport", rawBench)
rawBench = rawBench.set_index('ASONDATE').stack().reset_index().rename(columns={0:"VALUE"})

notEmpty


c:\users\mukesh\appdata\local\programs\python\python37\lib\site-packages\pandas\core\frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [3]:
allDateRawBench = pd.DataFrame(index=pd.date_range(rawBench['ASONDATE'].min(),rawBench['ASONDATE'].max()))
pivotRawBench = rawBench.pivot_table(index="ASONDATE",columns='PORTCODE',values="VALUE")

In [4]:
rawPrices = allDateRawBench.merge(pivotRawBench,left_index=True,right_index=True,how='left').fillna(method='ffill')
dividend = pd.DataFrame(index=rawPrices.index,columns=rawPrices.columns,data=0)
splitRatio = pd.DataFrame(index=rawPrices.index,columns=rawPrices.columns,data=1)

In [5]:
adjPrice = cu.calcAdjustedPrices(rawPrices,dividend,splitRatio).sort_index()
db.insertDataToMongo("dailyadjclose_navport",adjPrice.reset_index())

In [6]:
logReturns = np.log(adjPrice.pct_change() + 1).replace([np.inf, -np.inf], np.nan).fillna(0)
db.insertDataToMongo("dailyadjclose_navport",logReturns.reset_index())

In [7]:
adjPrice = adjPrice.stack().reset_index().rename(columns={'level_0':"ASONDATE","level_1":"PORTCODE",0:"ADJPRICES"})
logReturns = logReturns.stack().reset_index().rename(columns={'level_0':"ASONDATE","level_1":"PORTCODE",0:"LOGRETURNS"})

In [8]:
dailyclreturns_navport = adjPrice.merge(logReturns,on=['ASONDATE','PORTCODE'])
dailyclreturns_navport = dailyclreturns_navport.set_index(['ASONDATE','PORTCODE']).stack().reset_index().rename(columns={"level_2":"OPERATION",0:"VALUE"})

In [9]:
# db.insertDataToMongo("dailyclreturns_navport",dailyclreturns_navport)

In [10]:
dailyclreturns_navport = dailyclreturns_navport.pivot_table(index="ASONDATE", columns=["OPERATION", "PORTCODE"])
dailyclreturns_navport = dailyclreturns_navport.iloc[:, dailyclreturns_navport.columns.get_level_values(1) == "LOGRETURNS"].fillna(0)

In [11]:
dailyclreturns_navport.columns = dailyclreturns_navport.columns.droplevel(0)

In [12]:
groupReturns = cu.calcGroupReturns("2018-01-01", dailyclreturns_navport, ["LOGRETURNS"])
groupReturns['TYPE']="PORTFOLIO"
groupReturns['ASONDATE']="2018-01-01"

In [13]:
db.insertDataToMongo("groupreturns", groupReturns.reset_index(), "2018-05-01")

In [14]:
db.insertDataToOracle("groupreturns",groupReturns.reset_index(),type="PORTFOLIO")

In [16]:
import pyfolio as pyf

In [17]:
dailyclreturns_navport.columns = dailyclreturns_navport.columns.droplevel(0)
groupRatio = dailyclreturns_navport.apply(lambda x: pyf.timeseries.perf_stats(x))
db.insertDataToMongo("groupratio_navport",groupRatio)

c:\users\mukesh\appdata\local\programs\python\python37\lib\site-packages\empyrical\stats.py:1511: RuntimeWarning: invalid value encountered in double_scalars
  np.abs(np.percentile(returns, 5))
